Σε αυτό το πείραμα πειράζω τις δωρεάν άδειες ενός κλάδου και βλέπω τις αλλαγές στην τιμή της άδειας στο σ΄συστημα καθώς και τις μεταβολής των κερδών του κλάδου αυτόυ.

In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from funkwpap import sympy_to_gurobi, Firm, Country, Sector, Regulator, get_emission
import sympy as sp, pandas as pd, numpy as np, tqdm, time, sys
from gurobipy import Model, LinExpr, QuadExpr, GRB
x = sp.symbols('x')
y = sp.symbols('y')

In [2]:

# Create Sectors and Countries
Regulator2 = Regulator('EU', permit_price= 2, emission_cap= 100)
sector1 = Sector('cement', price_demand_function= 100 - 0.1*x, free_emission_multiplier= 0, regulator= Regulator2)
sector2 = Sector('steel', price_demand_function=150 - 0.1*x, free_emission_multiplier= 0, regulator= Regulator2)
sector3 = Sector('paper', price_demand_function= 200 - 0.02*x**1.5, free_emission_multiplier= 0, regulator= Regulator2)
sector4 = Sector('plastic', price_demand_function= 400 - 0.5*x, free_emission_multiplier= 0, regulator= Regulator2)
sector5 = Sector('glass', price_demand_function= 300 - 0.4*x, free_emission_multiplier= 0, regulator= Regulator2)
country1 = Country('DE', 1, regulator= Regulator2)
country2 = Country('FI', 0.5, regulator= Regulator2)
country3 = Country('GR', size= 0.1, regulator= Regulator2)

# Create Firms using objects
firm1 = Firm('firm1', 1, 1, x*0, 0.1*x**3 + 100/y**2, 0, 0, 0, regulator= Regulator2)
firm2 = Firm('firm2', 1, 2, x*0, 5*x**3.2 + 40/y**2, 0, 0, 0, regulator= Regulator2)
firm3 = Firm('firm3', 1, 3, x*0, x**5.2 + 30/y**2, 0, 0, 0, regulator= Regulator2)
firm4 = Firm('firm4', 2, 1, x*0, x**3.3 + 20/y**2, 0, 0, 0, regulator= Regulator2)
firm5 = Firm('firm5', 2, 2, x*0, x**2.3 + 20/y**2, 0, 0, 0, regulator= Regulator2)
firm6 = Firm('firm6', 2, 3, x*0, x**3.2 + 15/y**2, 0, 0, 0, regulator= Regulator2)
firm7 = Firm('firm7', 3, 1, x*0, x**4.1 + 40/y**2, 0, 0, 0, regulator= Regulator2)
firm8 = Firm('firm8', 3, 2, x*0, x**5.1 + 50/y**2, 0, 0, 0, regulator= Regulator2)
firm9 = Firm('firm9', 3, 3, x*0, x**5 + 60/y**2, 0, 0, 0, regulator= Regulator2)
firm10 = Firm('firm10', 4, 1, x*0, x**4.2 + 70/y**2, 0, 0, 0, regulator= Regulator2)
firm11 = Firm('firm11', 4, 2, x*0, x**3.1 + 80/y**2, 0, 0, 0, regulator= Regulator2)
firm12 = Firm('firm12', 4, 3, x*0, x**2.1 + 90/y**2, 0, 0, 0, regulator= Regulator2)
firm13 = Firm('firm13', 5, 1, x*0, x**2.1 + 100/y**2, 0, 0, 0, regulator= Regulator2)
firm14 = Firm('firm14', 5, 2, x*0, x**3.1 + 110/y**2, 0, 0, 0, regulator= Regulator2)
firm15 = Firm('firm15', 5, 3, x*0, x**2.1 + 120/y**2, 0, 0, 0, regulator= Regulator2)
firm16 = Firm('firm16', 1, 1, x*0, x**3.1 + 130/y**2, 0, 0, 0, regulator= Regulator2)
firm17 = Firm('firm17', 1, 2, x*0, x**4.1 + 140/y**2, 0, 0, 0, regulator= Regulator2)
firm18 = Firm('firm18', 1, 3, x*0, x**5.1 + 150/y**2, 0, 0, 0, regulator= Regulator2)
firm19 = Firm('firm19', 1, 1, x*0, x**5.1 + 160/y**2, 0, 0, 0, regulator= Regulator2)
firm20 = Firm('firm20', 2, 2, x*0, x**4.1 + 170/y**2, 0, 0, 0, regulator= Regulator2)
firm21 = Firm('firm21', 2, 3, x*0, x**3.1 + 180/y**2, 0, 0, 0, regulator= Regulator2)
firm22 = Firm('firm22', 2, 1, x*0, x**2.1 + 190/y**2, 0, 0, 0, regulator= Regulator2)
firm23 = Firm('firm23', 2, 2, x*0, x**3.1 + 200/y**2, 0, 0, 0, regulator= Regulator2)
firm24 = Firm('firm24', 3, 3, x*0, x**4.1 + 210/y**2, 0, 0, 0, regulator= Regulator2)
firm25 = Firm('firm25', 3, 1, x*0, x**5.1 + 220/y**2, 0, 0, 0, regulator= Regulator2)
firm26 = Firm('firm26', 3, 2, x*0, x**4.1 + 230/y**2, 0, 0, 0, regulator= Regulator2)
firm27 = Firm('firm27', 3, 3, x*0, x**3.1 + 240/y**2, 0, 0, 0, regulator= Regulator2)
firm28 = Firm('firm28', 4, 1, x*0, x**2.1 + 250/y**2, 0, 0, 0, regulator= Regulator2)
firm29 = Firm('firm29', 4, 2, x*0, x**3.1 + 260/y**2, 0, 0, 0, regulator= Regulator2)
firm30 = Firm('firm30', 4, 3, x*0, x**4.1 + 270/y**2, 0, 0, 0, regulator= Regulator2)




print(sector1.firms)  # List of firms in sector1
print(country1.firms)  # List of firms in country1
print(Regulator2.firm_registry)  # Dictionary of all firms

for firm in Regulator2.firm_registry.values():
    print(firm.name, firm.sector.name, firm.country.name)


[Firm(id=1, name='firm1', sector_id=1, country_id=1, actual_output=0, emission=0, profit=0), Firm(id=2, name='firm2', sector_id=1, country_id=2, actual_output=0, emission=0, profit=0), Firm(id=3, name='firm3', sector_id=1, country_id=3, actual_output=0, emission=0, profit=0), Firm(id=16, name='firm16', sector_id=1, country_id=1, actual_output=0, emission=0, profit=0), Firm(id=17, name='firm17', sector_id=1, country_id=2, actual_output=0, emission=0, profit=0), Firm(id=18, name='firm18', sector_id=1, country_id=3, actual_output=0, emission=0, profit=0), Firm(id=19, name='firm19', sector_id=1, country_id=1, actual_output=0, emission=0, profit=0)]
[Firm(id=1, name='firm1', sector_id=1, country_id=1, actual_output=0, emission=0, profit=0), Firm(id=4, name='firm4', sector_id=2, country_id=1, actual_output=0, emission=0, profit=0), Firm(id=7, name='firm7', sector_id=3, country_id=1, actual_output=0, emission=0, profit=0), Firm(id=10, name='firm10', sector_id=4, country_id=1, actual_output=0,

In [3]:
# Regulator2.optimize_them_all(precision = 0.1, print_diff = True, BAU = True)

In [4]:
for firm in Regulator2.firm_registry.values():
    print(firm.name, firm.sector.name, firm.country.name, firm.actual_output, firm.emission, firm.profit)

firm1 cement DE 0 0 0
firm2 cement FI 0 0 0
firm3 cement GR 0 0 0
firm4 steel DE 0 0 0
firm5 steel FI 0 0 0
firm6 steel GR 0 0 0
firm7 paper DE 0 0 0
firm8 paper FI 0 0 0
firm9 paper GR 0 0 0
firm10 plastic DE 0 0 0
firm11 plastic FI 0 0 0
firm12 plastic GR 0 0 0
firm13 glass DE 0 0 0
firm14 glass FI 0 0 0
firm15 glass GR 0 0 0
firm16 cement DE 0 0 0
firm17 cement FI 0 0 0
firm18 cement GR 0 0 0
firm19 cement DE 0 0 0
firm20 steel FI 0 0 0
firm21 steel GR 0 0 0
firm22 steel DE 0 0 0
firm23 steel FI 0 0 0
firm24 paper GR 0 0 0
firm25 paper DE 0 0 0
firm26 paper FI 0 0 0
firm27 paper GR 0 0 0
firm28 plastic DE 0 0 0
firm29 plastic FI 0 0 0
firm30 plastic GR 0 0 0


In [5]:
Regulator2.BAU_calculator(print_diff = True)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-14
Max diff: 0.00, permit price = 2.0, cap = 100, second_stage = False, a = 0.9000

In [6]:
# Let's see what happens in we change the free allocation of sector A keeping the cap at 80% of the BAU emissions
free_allocation_sectorA = list(np.linspace(0, 1, 20))
# Read the data from the csv file, if it exists, if not, just create a new one
try:
    df = pd.read_csv('../data/generated/exp4_how_Free_allocation_of_one_sector_affects_others_v0_1.csv')
    free_alloc = list(df['Free Allocation'])
    caps2 = list(df['Emission Cap'])
    permit_prices = list(df['Permit Price'])
    outputs = list(df['Outputs'])
    emissions = list(df['Emissions'])
    profits = list(df['Profits'])
except:
    free_alloc = []
    caps2 = []
    permit_prices = []
    outputs = []
    emissions = []
    profits = []
# Loop through all the emission caps and calculate the permit price

# Find all the caps that are not calculated yet (meaning that they are more than 0.5 different than a value of the caps2)
for i in free_allocation_sectorA[:]:  # Iterate over a copy of the list
    if any(abs(i - x) < 0.001 for x in free_alloc):
        free_allocation_sectorA.remove(i)

Regulator2.emission_cap = 0.8*Regulator2.BAU_emissions

for i in tqdm.tqdm(free_allocation_sectorA):
    Regulator2.sector_registry[1].free_emission_multiplier = i
    Regulator2.find_optimal_permit_price_to_meet_the_emission_cap_requirements(precision= 1, x_high = 1000000)
    caps2.append(Regulator2.emission_cap)
    free_alloc.append(i)
    permit_prices.append(Regulator2.permit_price)
    outputs.append(tuple([firm.actual_output for firm in Regulator2.firm_registry.values()]))
    emissions.append(tuple([firm.emission for firm in Regulator2.firm_registry.values()]))
    profits.append(tuple([firm.profit for firm in Regulator2.firm_registry.values()]))
    # save all the data in a csv file
    df = pd.DataFrame({'Free Allocation': free_alloc, 'Emission Cap': caps2, 'Permit Price': permit_prices, 'Outputs': outputs, 'Emissions': emissions, 'Profits': profits})
    # Save the DataFrame to the specified path
    df.to_csv('../data/generated/exp4_how_Free_allocation_of_one_sector_affects_others_v0_1.csv', index=False)

  0%|          | 0/20 [00:00<?, ?it/s]

Permit price: 29.087067price = 29.1, cap = 3075, second_stage = False, a = 1.00000000

  5%|▌         | 1/20 [08:04<2:33:26, 484.57s/it]

Permit price: 29.087066650390625 and total emission: 3068.1224210197265 and emission cap 3074.5378274769137
Permit price: 30.040741price = 30.0, cap = 3075, second_stage = False, a = 1.00000000

 10%|█         | 2/20 [16:33<2:29:36, 498.68s/it]

...

Permit price: 41.961669921875 and total emission: 3074.330531562796 and emission cap 3074.5378274769137
Permit price: 43.392181price = 43.4, cap = 3075, second_stage = False, a = 1.00000000

 95%|█████████▌| 19/20 [2:37:01<08:52, 532.56s/it]

Permit price: 43.392181396484375 and total emission: 3068.9731660300336 and emission cap 3074.5378274769137
Permit price: 44.345856price = 44.3, cap = 3075, second_stage = False, a = 1.00000000

100%|██████████| 20/20 [2:44:50<00:00, 494.55s/it]

Permit price: 44.345855712890625 and total emission: 3067.9099114207706 and emission cap 3074.5378274769137
